In [2]:
# here give error you should install spark in your device and conect with it   or same note book but in kaggle 
# https://www.kaggle.com/code/hussienawad/is-fraud-analysis

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/kaggle/working/spark-3.0.1-bin-hadoop3.2"
import findspark

findspark.init()
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder\
        .master("local")\
        .appName("Kaggle")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
sc = spark.sparkContext


The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'spark-3.0.1-bin-hadoop3.2.tgz'


Exception: Unable to find py4j in /kaggle/working/spark-3.0.1-bin-hadoop3.2\python, your SPARK_HOME may not be configured correctly

In [ ]:
sc

In [ ]:
spark

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("example").config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

In [ ]:
df = spark.read.csv("/content/fraudTrain.csv",inferSchema=True, header=True)
df.show(5)

In [ ]:
analysis, val_data, test_data = df.randomSplit([0.1, 0.7, 0.2], seed=42)

In [ ]:
import plotly.express as px
from pyspark.sql.functions import count
fraud_counts = analysis.groupBy("is_fraud").agg(count("*").alias("count"))

print(fraud_counts.collect())
# collect counts as list
fraud_or_not = [int(row["count"]) for row in fraud_counts.collect()]
labels = ["Fraud","notFraud"]

# create pie chart
fig = px.pie(values=fraud_or_not, names=labels, width=700, height=400, color_discrete_sequence=["skyblue","black"], title="notFraud vs Fraud")
fig.show()


In [ ]:
counts = analysis.groupBy("gender").agg(count("*").alias("count"))
print(counts.collect())
# collect counts as list
count = [int(row["count"]) for row in counts.collect()]
labels = ["F","M"]

# create pie chart
fig = px.pie(values=count, names=labels, width=700, height=400, color_discrete_sequence=["skyblue","black"], title="F vs M")
fig.show()

In [ ]:
unique_values = analysis.select("city").distinct().collect()
unique_values_list = [row["city"] for row in unique_values]
print(len(unique_values_list))
unique_values_list

In [ ]:
from pyspark.sql.functions import to_timestamp
df = df.withColumn('timestamp_col', to_timestamp('trans_date_trans_time', 'yyyy-MM-dd HH:mm:ss'))


In [ ]:
from pyspark.sql.functions import max, min

# assume you have a dataframe df with a column 'datetime_col' of datetime type
max_date = df.select(max('timestamp_col')).collect()[0][0]
min_date = df.select(min('timestamp_col')).collect()[0][0]

print("Maximum datetime value:", max_date)
print("Minimum datetime value:", min_date)

In [ ]:
from pyspark.sql.functions import col

# Assuming your DataFrame is named 'df', you can get unique values for columns 'col1' and 'col2' as follows
unique_values = analysis.select(col("first"), col("last")).distinct()
unique_values.show()

In [ ]:
unique_values.count()

In [ ]:
from pyspark.sql.functions import col
import plotly.express as px
from pyspark.sql.functions import count
df_filtered = df.filter(col('is_fraud') ==1)
col2_extracted = df_filtered.select('amt')
col2_extracted.agg(count("*").alias("count")).collect()

In [ ]:
from pyspark.sql.functions import sum
col2_extracted.agg(sum("amt")).collect()[0][0]

In [ ]:
col_rdd = col2_extracted.rdd.map(lambda row: row[0])

# define function to map each partition of RDD to a tuple of (sum, count)
def map_partition(iter):
    sum = 0
    count = 0
    for val in iter:
        sum += val
        count += 1
        if count == 5000:
            yield (sum, count)
            sum = 0
            count = 0
    if count > 0:
        yield (sum, count)

# map each partition of RDD to a tuple of (sum, count)
mapped_rdd = col_rdd.mapPartitions(map_partition)

# reduce by summing the sum values and counting the count values
(sum, count) = mapped_rdd.reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# calculate mean
mean = sum / count

print("Mean: ", mean)


In [ ]:
amt = analysis.select(col("amt"))
amt.show(5)

In [ ]:
col_rdd = amt.rdd.map(lambda row: row[0])

# define function to map each partition of RDD to a tuple of (sum, count)
def map_partition(iter):
    sum = 0
    count = 0
    for val in iter:
        sum += val
        count += 1
        if count == 10000:
            yield (sum, count)
            sum = 0
            count = 0
    if count > 0:
        yield (sum, count)

# map each partition of RDD to a tuple of (sum, count)
mapped_rdd = col_rdd.mapPartitions(map_partition)

# reduce by summing the sum values and counting the count values
(sum, count) = mapped_rdd.reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# calculate mean
mean = sum / count

print("Mean: ", mean)



In [ ]:
df.printSchema()

In [ ]:
deduped_df = df.dropDuplicates()

In [ ]:
import pyspark.sql.functions as fn
from pyspark.sql.functions import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [ ]:
def stringIndexer(colName,newName,df):
  stringIndexer = StringIndexer(inputCol=colName, outputCol=newName)
  model = stringIndexer.fit(df)
  indexed = model.transform(df)
  return indexed

In [ ]:
def oneHotEncoder(colName,newName,df):
  encoder = OneHotEncoder(inputCol=colName, outputCol=newName)
  encoded = encoder.fit(df).transform(df)
  return encoded

In [ ]:
df_m=stringIndexer("merchant","merchantIndex",deduped_df)
df_m=oneHotEncoder("merchantIndex","merchantVec",df_m)
df_m=stringIndexer("category","categoryIndex",df_m)
df_m=oneHotEncoder("categoryIndex","categoryVec",df_m)
df_m=stringIndexer("gender","genderIndex",df_m)
df_m=oneHotEncoder("genderIndex","genderVec",df_m)
df_m=stringIndexer("job","jobIndex",df_m)
df_m=oneHotEncoder("jobIndex","jobVec",df_m)

In [ ]:
df_m.show(3)

In [ ]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import to_timestamp, hour



In [ ]:
df_m = df_m.withColumn('timestamp_col', to_timestamp('trans_date_trans_time', 'yyyy-MM-dd HH:mm:ss'))
df_m = df_m.withColumn('hour_col', hour('timestamp_col'))

choose col use to train


In [ ]:
assembler = VectorAssembler(inputCols=["amt", "zip", "city_pop","unix_time","merch_lat","merch_long","hour_col","merchantVec","categoryVec","genderVec","jobVec"], outputCol="features")

X = assembler.transform(df_m)

In [ ]:
X.show(5)

In [ ]:
trainDF,testDF = X.randomSplit([.8,.2],seed=42)

In [ ]:
fea_Train=trainDF.select(col("amt"),col("zip"),col("city_pop"),col("unix_time"),col("merch_lat"),col("merch_long"),col("hour_col"),col("merchantIndex"),col("categoryIndex"),col("genderIndex"),col("jobIndex"),col("is_fraud"))
fea_test=testDF.select(col("amt"),col("zip"),col("city_pop"),col("unix_time"),col("merch_lat"),col("merch_long"),col("hour_col"),col("merchantIndex"),col("categoryIndex"),col("genderIndex"),col("jobIndex"),col("is_fraud"))

In [ ]:

train_rdd = fea_Train.rdd.map(lambda row: list(row))
# Define a function to compute the distance between two points
def distance(x, y):
    e=0
    for i in range(len(x)-1):
        e+=(float(x[i]) - float(y[i]))**2 
    return e

# Define a function to map each test point to its k nearest neighbors in the training data
def map_partition(iter, param):
    # count=0
    # for i in iter:
    #   count+=1
    #   if count==10:

    #     yield (i, "eee")
    #     count=0
    # if count>0:
    #   yield ("iter", "eee")
    
    count = 0
    l={}
    
    for val in iter:
        
          d=distance(t,val)
          l[d]=val[11]
    
          count += 1
          if count == 10000:
              sorted_list = sorted(l.keys())[:5]
              out={}
              for s in sorted_list:
                out[s]=l[s]
              yield (param, out)
              count = 0
    sorted_list = sorted(l.keys())[:5]
    out={}
    for s in sorted_list:
      out[s]=l[s]
    if count >0:
        yield (param, out)
def knn_reducer(x, y):
    r={}
    if x[0]==y[0]:
      x[1].update(y[1])

    return x
# Define a function to reduce the nearest neighbors from each partition
# def knn_reducer(x, y):
#     merged = sorted(x + y)[:k]
#     return merged



# Map each test data point to its k nearest neighbors in the training data
predicted=[]
truePred=[]
for t in fea_test.collect():
  
  mapped_rdd = train_rdd.mapPartitions(lambda iterator: map_partition(iterator, t))
  #print(mapped_rdd.collect())
  sum = mapped_rdd.reduce(lambda x, y: (knn_reducer(x, y)))
  
  
  sorted_list = sorted(sum[1])
  predicted.append(sum[1][sorted_list[0]])
  truePred.append(t[11])
  
 
# Reduce the nearest neighbors from each partition
# neighbors = neighbors_rdd.reduce(lambda x, y: knn_reducer(x, y))

# # Print the k nearest neighbors and their distances
# print(neighbors)


In [ ]:
import numpy as np
from sklearn.metrics import classification_report
print(np.sum(truePred))

print(classification_report(predicted,truePred))

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
rf = RandomForestClassifier(labelCol="is_fraud", featuresCol="features", numTrees=10)


In [ ]:
model = rf.fit(trainDF)


In [ ]:
predictions = model.transform(testDF)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)


In [ ]:
# Calculate precision
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
print("Precision: ", precision)

# Calculate recall
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
print("Recall: ", recall)

# Calculate F1 score
f1score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
print("F1 Score: ", f1score)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC

from pyspark.sql.functions import col


svm = LinearSVC(labelCol="is_fraud", featuresCol="features",maxIter=10, regParam=0.1)
svm_model = svm.fit(trainDF)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions = svm_model.transform(testDF)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

In [ ]:
# Calculate precision
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
print("Precision: ", precision)

# Calculate recall
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
print("Recall: ", recall)

# Calculate F1 score
f1score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
print("F1 Score: ", f1score)

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="is_fraud", featuresCol="features")
model = lr.fit(trainDF)

In [ ]:
predictions = model.transform(testDF)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

In [ ]:
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
print("Precision: ", precision)

# Calculate recall
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
print("Recall: ", recall)

# Calculate F1 score
f1score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})
print("F1 Score: ", f1score)